In [1]:
# =============================================
# 🔁 BATCH RUNNER — MULTIPLE C1 INDICATORS
# =============================================

import os
import yaml
import pandas as pd
import shutil
from datetime import datetime
from backtester import run_backtest
from utils import summarize_results

# =============================================
# 🔧 SETUP
# =============================================
with open("batch_config.yaml", "r") as f:
    batch_config = yaml.safe_load(f)

indicator_dict = batch_config["batch_test"]["indicators"]
default_params = batch_config["batch_test"].get("default_params", {})
STARTING_BALANCE = 10000
RESULTS_CSV = "results/c1_batch_results.csv"

# === Utility to load and save config ===
def load_config(path="config.yaml"):
    with open(path, "r") as f:
        return yaml.safe_load(f)

def save_config(config, path="config.yaml"):
    with open(path, "w") as f:
        yaml.dump(config, f)


ModuleNotFoundError: No module named 'backtester'

In [2]:
# =============================================
# 🚀 RUN BATCH TESTS
# =============================================
all_results = []

for short_name, custom_params in indicator_dict.items():
    print(f"\\n🔍 Testing C1: {short_name}")

    # Combine default and custom params
    params = default_params.copy()
    params.update(custom_params or {})

    full_name = f"c1_{short_name}"

    # --- Step A: Update config.yaml ---
    config = load_config()
    config["indicators"]["c1"] = short_name
    config["indicator_params"] = {full_name: params}
    save_config(config)

    # --- Step B: Run backtest ---
    run_backtest()

    # --- Step C: Archive result files ---
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    folder = f"results/results_history/{short_name}_{timestamp}"
    os.makedirs(folder, exist_ok=True)

    for fname in ["trades.csv", "summary.txt", "equity_curve.csv"]:
        if os.path.exists(f"results/{fname}"):
            shutil.copy(f"results/{fname}", os.path.join(folder, fname))

    # --- Step D: Summarize results ---
    trades_path = "results/trades.csv"

    if os.path.exists(trades_path) and os.path.getsize(trades_path) > 0:
        try:
            trades_df = pd.read_csv(trades_path)
        except pd.errors.EmptyDataError:
            print(f"⚠️ {short_name}: trades.csv is empty (no columns)")
            continue
    else:
        print(f"⚠️ {short_name}: trades.csv not found or has no content")
        continue

    params_str = ", ".join(f"{k}={v}" for k, v in params.items())

    metrics = summarize_results(
        trades=trades_df.to_dict("records"),
        starting_balance=STARTING_BALANCE,
        run_name=short_name,
        params=params_str
    )

    if metrics:
        metrics["indicator"] = short_name
        all_results.append(metrics)


NameError: name 'indicator_dict' is not defined

In [3]:
# =============================================
# 💾 SAVE FINAL RESULTS CSV
# =============================================
if all_results:
    df_results = pd.DataFrame(all_results)
    df_results.to_csv(RESULTS_CSV, index=False)
    print(f"\n✅ Saved C1 batch results to {RESULTS_CSV}")
else:
    print("⚠️ No results to save.")


⚠️ No results to save.
